In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response

load_dotenv()
logging.langsmith("CH03-OutputParser-MJ")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser-MJ


# 1. PydanticOutputParser

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [4]:
email_conversation = """보낸사람: 최성연 <snucsy@snu.ac.kr>
날짜: 2025년 2월 25일 (화) 오후 4:52
제목: RE: 소프트웨어 저작물 권리승계 관련 문의드립니다.
받는사람: 최재훈 <hoon95@snu.ac.kr>


최재훈 선생님, 안녕하세요.

시스템 권한이 부여되지 않았거나, '행정 직원'의 권한이 부여되었거나 등의 사유로 확인되지 않는 것으로 생각됩니다.

첨부한 서류를 신민정 선생님께서 작성/날인하신 뒤 회신 주시면 갈음하여 처리하겠습니다.

감사합니다.
최성연 드림


최성연 Choi Seongyeon
"""

In [9]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

이메일의 중요한 내용은 다음과 같습니다:

- 시스템 권한 문제로 인해 확인이 되지 않음.
- 첨부된 서류를 신민정 선생님이 작성 및 날인 후 회신 요청.

In [10]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [11]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [13]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

prompt = prompt.partial(format=parser.get_format_instructions())

In [14]:
chain = prompt | llm  

In [15]:
response = chain.stream(
    {
        "question": "이메일의 내용을 요약해주세요.",
        "email_conversation": email_conversation,
    }
)  

output = stream_response(response, return_output=True)

```json
{
    "person": "최성연",
    "email": "snucsy@snu.ac.kr",
    "subject": "RE: 소프트웨어 저작물 권리승계 관련 문의드립니다.",
    "summary": "시스템 권한 문제로 확인이 안 되고 있으며, 첨부한 서류를 신민정 선생님께서 작성 및 날인 후 회신해 주시면 처리하겠다는 내용입니다.",
    "date": "2025년 2월 25일"
}
```

In [17]:
structured_output = parser.parse(output)
print(structured_output)

person='최성연' email='snucsy@snu.ac.kr' subject='RE: 소프트웨어 저작물 권리승계 관련 문의드립니다.' summary='시스템 권한 문제로 확인이 안 되고 있으며, 첨부한 서류를 신민정 선생님께서 작성 및 날인 후 회신해 주시면 처리하겠다는 내용입니다.' date='2025년 2월 25일'


In [18]:
chain = prompt | llm | parser

In [20]:
response = chain.invoke(
    {
        "question": "이메일의 주요 내용을 추출해주세요.",
        "email_conversation": email_conversation,
    }
)

response

EmailSummary(person='최성연', email='snucsy@snu.ac.kr', subject='RE: 소프트웨어 저작물 권리승계 관련 문의드립니다.', summary='시스템 권한 문제로 확인이 안 되고 있으며, 첨부한 서류를 작성/날인하여 회신해 주시면 처리하겠다는 내용입니다.', date='')